### Content Based Recommendations

In the previous notebook, you were introduced to a way of making recommendations using collaborative filtering.  However, using this technique there are a large number of users who were left without any recommendations at all.  Other users were left with fewer than the ten recommendations that were set up by our function to retrieve....

In order to help these users out, let's try another technique: **content based** recommendations. Let's start off where we were in the previous notebook.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from IPython.display import HTML
import progressbar
import tests as t
import pickle


%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']


all_recs = pickle.load(open("all_recs.p", "rb"))

### Datasets

From the above, you now have access to three important items that you will be using throughout the rest of this notebook.  

`a.` **movies** - a dataframe of all of the movies in the dataset along with other content related information about the movies (genre and date)


`b.` **reviews** - this was the main dataframe used before for collaborative filtering, as it contains all of the interactions between users and movies.


`c.` **all_recs** - a dictionary where each key is a user, and the value is a list of movie recommendations based on collaborative filtering

For the individuals in **all_recs** who did receive 10 recommendations using collaborative filtering, we don't really need to worry about them.  However, there were a number of individuals in our dataset who did not receive any recommendations.

-----

`1.` Let's start with finding all of the users in our dataset who didn't get all 10 ratings we would have liked them to have using collaborative filtering.  

In [3]:
users_with_all_recs = [] # Store user ids who have all their recommendations in this (10 or more)
users_who_need_recs = []# Store users who still need recommendations here
for k,v in all_recs.items():
    if len(v)>=10:
        users_with_all_recs.append(k)
    else:
        users_who_need_recs.append(k)

In [4]:
# A quick test
assert len(users_with_all_recs) == 22187
print("That's right there were still another 31781 users who needed recommendations when we only used collaborative filtering!")

That's right there were still another 31781 users who needed recommendations when we only used collaborative filtering!


### Content Based Recommendations

You will be doing a bit of a mix of content and collaborative filtering to make recommendations for the users this time.  This will allow you to obtain recommendations in many cases where we didn't make recommendations earlier.     

`2.` Before finding recommendations, rank the user's ratings from highest to lowest. You will move through the movies in this order looking for other similar movies.

In [5]:
# create a dataframe similar to reviews, but ranked by rating for each user
ranked_review=reviews.sort_values(['user_id','rating'],ascending=[True,False])
ranked_review.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
16,2,1798709,10,1389948338,2014-01-17 08:45:38,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
18,2,2024544,10,1389355949,2014-01-10 12:12:29,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
22,2,2726560,9,1438293988,2015-07-30 22:06:28,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


### Similarities

In the collaborative filtering sections, you became quite familiar with different methods of determining the similarity (or distance) of two users.  We can perform similarities based on content in much the same way.  

In many cases, it turns out that one of the fastest ways we can find out how similar items are to one another (when our matrix isn't totally sparse like it was in the earlier section) is by simply using matrix multiplication.  If you are not familiar with this, an explanation is available [here by 3blue1brown](https://www.youtube.com/watch?v=LyGKycYT2v0) and another quick explanation is provided [in the post here](https://math.stackexchange.com/questions/689022/how-does-the-dot-product-determine-similarity).

For us to pull out a matrix that describes the movies in our dataframe in terms of content, we might just use the indicator variables related to **year** and **genre** for our movies.  

Then we can obtain a matrix of how similar movies are to one another by taking the dot product of this matrix with itself.  Notice below that the dot product where our 1 values overlap gives a value of 2 indicating higher similarity.  In the second dot product, the 1 values don't match up.  This leads to a dot product of 0 indicating lower similarity.

<img src="images/dotprod1.png" alt="Dot Product" height="500" width="500">

We can perform the dot product on a matrix of movies with content characteristics to provide a movie by movie matrix where each cell is an indication of how similar two movies are to one another.  In the below image, you can see that movies 1 and 8 are most similar, movies 2 and 8 are most similar, and movies 3 and 9 are most similar for this subset of the data.  The diagonal elements of the matrix will contain the similarity of a movie with itself, which will be the largest possible similarity (and will also be the number of 1's in the movie row within the orginal movie content matrix).

<img src="images/moviemat.png" alt="Dot Product" height="500" width="500">


`3.` Create a numpy array that is a matrix of indicator variables related to year (by century) and movie genres by movie.  Perform the dot product of this matrix with itself (transposed) to obtain a similarity matrix of each movie with every other movie.  The final matrix should be 31245 x 31245.

In [6]:
movies

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31240,8335880,Loro 2 (2018),Biography,2018,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31241,8342748,Hari Kondabolu: Warn Your Relatives (2018),Comedy,2018,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
31242,8342946,Tig Notaro: Happy To Be Here (2018),Comedy,2018,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
31243,8402090,Cumali Ceber 2 (2018),Comedy,2018,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [7]:
# Subset so movie_content is only using the dummy variables for each genre and the 3 century based year dummy columns
movie_content=movies.iloc[:,4:]
# Take the dot product to obtain a movie x movie matrix of similarities
dot_prod_movies=movie_content.dot(np.transpose(movie_content))

In [8]:
# create checks for the dot product matrix
assert dot_prod_movies.shape[0] == 31245
assert dot_prod_movies.shape[1] == 31245
assert dot_prod_movies.iloc[0, 0] == np.max(dot_prod_movies[0])
print("Looks like you passed all of the tests.  Though they weren't very robust - if you want to write some of your own, I won't complain!")

Looks like you passed all of the tests.  Though they weren't very robust - if you want to write some of your own, I won't complain!


In [9]:
dot_prod_movies.head(11)

,0,1,2,3,4,5,6,7,8,9,...,31235,31236,31237,31238,31239,31240,31241,31242,31243,31244
0,3,3,3,1,2,1,1,0,1,1,...,1,0,0,1,0,0,0,0,0,0
1,3,3,3,1,2,1,1,0,1,1,...,1,0,0,1,0,0,0,0,0,0
2,3,3,3,1,2,1,1,0,1,1,...,1,0,0,1,0,0,0,0,0,0
3,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,2,2,1,3,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
5,1,1,1,0,1,4,2,1,2,2,...,0,0,0,0,0,0,0,0,0,0
6,1,1,1,0,1,2,4,1,3,2,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
8,1,1,1,0,1,2,3,1,3,2,...,0,0,0,0,0,0,0,0,0,0
9,1,1,1,0,1,2,2,1,2,3,...,0,0,0,0,0,0,0,0,0,1


### For Each User...


Now you have a matrix where each user has their ratings ordered.  You also have a second matrix where movies are each axis, and the matrix entries are larger where the two movies are more similar and smaller where the two movies are dissimilar.  This matrix is a measure of content similarity. Therefore, it is time to get to the fun part.

For each user, we will perform the following:

    i. For each movie, find the movies that are most similar that the user hasn't seen.

    ii. Continue through the available, rated movies until 10 recommendations or until there are no additional movies.

As a final note, you may need to adjust the criteria for 'most similar' to obtain 10 recommendations.  As a first pass, I used only movies with the highest possible similarity to one another as similar enough to add as a recommendation.

`3.` In the cell below, complete each of the functions needed for making content based recommendations.

In [10]:
mivierow=movies.index[movies['movie_id']==91][0]
mivierow

4

In [11]:
np.where(movies['movie_id']==91)[0][0]

4

In [12]:
max(dot_prod_movies[mivierow].drop(mivierow))

2

In [13]:
dot_prod_movies[mivierow].drop(mivierow).index[dot_prod_movies[mivierow].drop(mivierow)==max(dot_prod_movies[mivierow].drop(mivierow))]

Int64Index([    0,     1,     2,    10,  1316,  2975,  3390,  5035,  7989,
             8803,  9245,  9422,  9584,  9670,  9854, 10433, 12766, 13735,
            13894, 14328, 14771, 15991, 17054, 17059, 17228, 17694, 17984,
            18026, 18309, 19490, 20942, 20958, 21044, 21308, 21563, 21597,
            21616, 21723, 21925, 22335, 22508, 23616, 23742, 23779, 23985,
            23997, 23998, 24138, 24322, 24406, 24473, 24527, 24579, 24663,
            24924, 24957, 25029, 25111, 25648, 25946, 26003, 26071, 26123,
            26225, 26407, 26445, 26489, 26675, 27177, 27441, 27720, 27888,
            27963, 28779, 28887, 29295, 29310, 30022, 30755, 30942, 30974,
            31102],
           dtype='int64')

In [14]:
# exclude the movie wer are trying to predict first, then find the dot product == max(dot product) of the rest of data
excludeid=dot_prod_movies[mivierow].drop(mivierow)
movieindex=excludeid.index[(excludeid==max(excludeid)) & (excludeid!=0)]
movieindex

Int64Index([    0,     1,     2,    10,  1316,  2975,  3390,  5035,  7989,
             8803,  9245,  9422,  9584,  9670,  9854, 10433, 12766, 13735,
            13894, 14328, 14771, 15991, 17054, 17059, 17228, 17694, 17984,
            18026, 18309, 19490, 20942, 20958, 21044, 21308, 21563, 21597,
            21616, 21723, 21925, 22335, 22508, 23616, 23742, 23779, 23985,
            23997, 23998, 24138, 24322, 24406, 24473, 24527, 24579, 24663,
            24924, 24957, 25029, 25111, 25648, 25946, 26003, 26071, 26123,
            26225, 26407, 26445, 26489, 26675, 27177, 27441, 27720, 27888,
            27963, 28779, 28887, 29295, 29310, 30022, 30755, 30942, 30974,
            31102],
           dtype='int64')

In [15]:
top10=movies.iloc[movieindex]['movie'].values.tolist()[:10]
top10

['Edison Kinetoscopic Record of a Sneeze (1894)',
 'La sortie des usines Lumière (1895)',
 'The Arrival of a Train (1896)',
 'Frankenstein (1910)',
 'Le sang des bêtes (1949)',
 'The Big Shave (1968)',
 'The Grandmother (1970)',
 'Zánik domu Usheru (1982)',
 'My Sweet Satan (1994)',
 'Ghosts (1997)']

In [16]:
def find_similar_movies(movie_id):
    '''
    INPUT
    movie_id - a movie_id 
    OUTPUT
    similar_movies - an array of the most similar movies by title
    '''
    # find the row of each movie id
    mivierow=np.where(movies['movie_id']==movie_id)[0][0]
    
    # find the most similar movie indices - to start I said they need to be the same for all content
    # exclude the movie wer are trying to predict first, then find the dot product == max(dot product) of the rest of data
    excludeid=dot_prod_movies[mivierow].drop(mivierow)
    movieindex=excludeid.index[(excludeid==max(excludeid)) & (excludeid!=0)]
    # pull the movie titles based on the indices
    similar_movies=movies.loc[movieindex]['movie'].values.tolist()[:10]
    
    return similar_movies
    
# You made this function in an earlier notebook - using again here    
def get_movie_names(movie_ids):
    '''
    INPUT
    movie_ids - a list of movie_ids
    OUTPUT
    movies - a list of movie names associated with the movie_ids
    
    '''
    movie_lst = list(movies[movies['movie_id'].isin(movie_ids)]['movie'])
   
    return movie_lst

In [17]:
find_similar_movies(91)

['Edison Kinetoscopic Record of a Sneeze (1894)',
 'La sortie des usines Lumière (1895)',
 'The Arrival of a Train (1896)',
 'Frankenstein (1910)',
 'Le sang des bêtes (1949)',
 'The Big Shave (1968)',
 'The Grandmother (1970)',
 'Zánik domu Usheru (1982)',
 'My Sweet Satan (1994)',
 'Ghosts (1997)']

In [18]:
get_movie_names([91])

['Le manoir du diable (1896)']

In [19]:
ranked_review.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
16,2,1798709,10,1389948338,2014-01-17 08:45:38,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
18,2,2024544,10,1389355949,2014-01-10 12:12:29,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
22,2,2726560,9,1438293988,2015-07-30 22:06:28,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [20]:
recs={}
users=[1]
for user in users:
    recs[user]=[]
    for watchedmovie in ranked_review.query("user_id==@user")['movie_id']:
        if len(recs[user])>= 10:
            break
        print('~~~')
        recomands = find_similar_movies(watchedmovie)
        for eachone in recomands:
            if len(recs[user])<10:
                recs[user].append(eachone)
recs

~~~


{1: ["Fantômas - À l'ombre de la guillotine (1913)",
  'Regeneration (1915)',
  'Schloß Vogeloed (1921)',
  'The Lodger: A Story of the London Fog (1927)',
  'Underworld (1927)',
  'Die Büchse der Pandora (1929)',
  'The Docks of New York (1928)',
  'The Big House (1930)',
  'Escape! (1930)',
  'Hogaraka ni ayume (1930)']}

In [21]:
len(ranked_review['user_id'].unique())

53968

In [28]:
def make_recs():
    '''
    INPUT
    None
    OUTPUT
    recs - a dictionary with keys of the user and values of the recommendations
    '''
    # Create dictionary to return with users and ratings
    recs={}
    # How many users for progress bar
    n_users= len(ranked_review['user_id'].unique())
    
    # Create the progressbar
    cnter=0
    bar = progressbar.ProgressBar(maxval=n_users+1, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    # For each user
        
        # Update the progress bar


        # Pull only the reviews the user has seen

        # Look at each of the movies (highest ranked first), 
        # pull the movies the user hasn't seen that are most similar
        # These will be the recommendations - continue until 10 recs 
        # or you have depleted the movie list for the user
    
    # loop each user
    for user in ranked_review['user_id'].unique():
        cnter+=1
        bar.update(cnter)
        # initial empty move list for each user
        recs[user]=[]
        # loopr each move target user watched already
        for watchedmovie in ranked_review.query("user_id==@user")['movie_id']:
            if len(recs[user])>= 10:
                break
            # find top10 movie to recommend
            recomands = find_similar_movies(watchedmovie)
            # append each movie into movie list
            for eachone in recomands:
                if len(recs[user])<10:
                    recs[user].append(eachone)


    # finish the progress bar

    bar.finish()
    return recs

In [29]:
recs = make_recs()

[========================================================================] 100%


### How Did We Do?

Now that you have made the recommendations, how did we do in providing everyone with a set of recommendations?

`4.` Use the cells below to see how many individuals you were able to make recommendations for, as well as explore characteristics about individuals for whom you were not able to make recommendations.  

In [30]:
# Some characteristics of my content based recommendations
users_without_all_recs = [k for k,v in recs.items() if len(v) <10] #store user ids without recs
users_with_all_recs = [k for k,v in recs.items() if len(v) ==10] # users with all recs here
no_recs = [k for k,v in recs.items() if len(v) ==0] # users with no recommendations

print("There were {} users without all 10 recommendations we would have liked to have.".format(len(users_without_all_recs)))
print("There were {} users with all 10 recommendations we would like them to have.".format(len(users_with_all_recs)))
print("There were {} users with no recommendations at all!".format(len(no_recs)))

There were 1994 users without all 10 recommendations we would have liked to have.
There were 51974 users with all 10 recommendations we would like them to have.
There were 0 users with no recommendations at all!


In [25]:
# Closer look at individual user characteristics - this may help it was from an earlier notebook
user_items = reviews[['user_id', 'movie_id', 'rating']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

def movies_watched(user_id):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    OUTPUT:
    movies - an array of movies the user has watched
    '''
    movies = user_by_movie.loc[user_id][user_by_movie.loc[user_id].isnull() == False].index.values

    return movies


movies_watched(189)

array([457430], dtype=int64)

In [32]:
recs[189]

['The Lord of the Rings: The Fellowship of the Ring (2001)',
 'The Legend of Bagger Vance (2000)',
 'Behind Enemy Lines (2001)',
 'Rules of Engagement (2000)',
 'The Lord of the Rings: The Return of the King (2003)',
 'The Lord of the Rings: The Two Towers (2002)',
 'Tigerland (2000)',
 'Frequency (2000)',
 'Ginger Snaps (2000)',
 'Malèna (2000)']

In [31]:
# More exploring
print("Some of the movie lists for users without any recommendations include:")
# Just one of many questions you could continue exploring...

Some of the movie lists for users without any recommendations include:


### Now What?  

Well, if you were really strict with your criteria for how similar two movies are (like I was initially), then you still have some users that don't have all 10 recommendations (and a small group of users who have no recommendations at all). 

As stated earlier, recommendation engines are a bit of an **art** and a **science**.  There are a number of things we still could look into - how do our collaborative filtering and content based recommendations compare to one another? How could we incorporate user input along with collaborative filtering and/or content based recommendations to improve any of our recommendations?  How can we truly gain recommendations for every user?

`5.` In this last step feel free to explore any last ideas you have with the recommendation techniques we have looked at so far.  You might choose to make the final needed recommendations using the first technique with just top ranked movies.  You might also loosen up the strictness in the similarity needed between movies.  Be creative and share your insights with your classmates!

In [ ]:
# Cells for exploring